<h2>3D stack to 2D MIP - Single image - Marker+ based on colocalization</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) into a MIP and allows you to:

1. Inspect your images in Napari.
2. Define regions of interest (ROIs) using labels in Napari. Store said ROIs as .tiff files if needed.
3. Predict nuclei labels and store them as .tiff files for further processing.
4. Extract numbers of cells positive for a marker based on colocalization (using a user-defined threshold).
5. Display positive cells in Napari.
6. Extract and save number of positive cells in a .csv file (marker_+_label_coloc_2D).

In [9]:
from pathlib import Path
from tqdm import tqdm
import tifffile
import napari
import os
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
from utils import list_images, read_image, save_rois, segment_nuclei_2d, segment_marker_positive_nuclei

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [10]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds"

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>. By changing the <code>slicing factor</code> you lose resolution but speed up processing times (check the results).

In [11]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[0]

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Generate maximum intensity projection and extract filename
img_mip, filename = read_image(image, slicing_factor)

# Show image in Napari to define ROI
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

Image analyzed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)


<Image layer 'img_mip' at 0x1f368daa910>

<h3>Label your regions of interest in Napari and explore the signal of your marker of interest</h3>

Make sure to set <code>n edit dim = 3</code> so the label propagates across all channels. Name your regions of interest as i.e. <code>DG</code>, <code>CA1</code>, <code>CA3</code> or <code>HIPPO</code>. If you do not draw any ROI the entire image will be analyzed.

Fnally, select the <code>img_mip</code> layer and play with the contrast limit to later set a threshold above which cells will be considered positive for said marker.

<video controls>
  <source src="./assets/napari_labels.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

<h3>Save user-defined label ROIs as .tiff files</h3>

In [12]:
save_rois(viewer, directory_path, filename)

No user-defined ROIs have been stored


In [13]:
# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

The following regions of interest will be analyzed: ['ALL', 'CA', 'DG']


<h3>Define your nuclei/marker stacks and your marker/erosion thresholds</h3>

Modify the values for <code>nuclei_channel</code>, <code>marker_channel</code>, <code>nuclei_channel_threshold</code> and <code>erosion_factor</code>

Define marker <code>location</code> as either "cytoplasm" or "nucleus", and the <code>cytoplasm_dil_radius</code>.

In [14]:
# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3
marker_channel = 1

# Define the intensity threshold above which a cell is considered positive for a marker
marker_channel_threshold = 40

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0

# Sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+
erosion_factor = 4

# Define marker location ("nucleus or "cytoplasm")
location = "nucleus"

# Define the amount of pixels you want to add around the nuclei to simulate the cytoplasm
cytoplasm_dilation_radius = 2

# Slice the nuclei and marker stack
nuclei_img = img_mip[nuclei_channel, :, :]
marker_img = img_mip[marker_channel, :, :]

<h3>Mask the input image with the user defined labels and extract data</h3>

In [15]:
# Create an empty list to store all stats extracted from each image
stats = []

for roi_name in tqdm(roi_names):

    # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
    try:
        # Read previously defined ROIs
        user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

    except FileNotFoundError:
        # Extract the xy dimensions of the input image 
        img_shape = img_mip.shape
        img_xy_dims = img_shape[-2:]

        # Create a label covering the entire image
        user_roi = np.ones(img_xy_dims).astype(np.uint8)

    # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
    try:
        # Read the nuclei predictions per ROI
        nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")

    except FileNotFoundError:

        # Slice the nuclei stack
        nuclei_img = img_mip[nuclei_channel, :, :]

        # We will create a mask where roi is greater than or equal to 1
        mask = (user_roi >= 1).astype(np.uint8)

        # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
        masked_nuclei_img = np.where(mask, nuclei_img, 0)

        # Segment nuclei and return labels
        nuclei_labels = segment_nuclei_2d(masked_nuclei_img, gaussian_sigma, cellpose_nuclei_diameter)

        # Save nuclei labels as .tiff files to reuse them later
        # Create nuclei_predictions directory if it does not exist
        try:
            os.makedirs(directory_path / "nuclei_preds" / roi_name)
        except FileExistsError:
            pass

        # Construct path to store
        path_to_store = directory_path / "nuclei_preds" / roi_name / f"{filename}.tiff"

        # Save mask (binary image)
        tifffile.imwrite(path_to_store, nuclei_labels)

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(nuclei_labels, name=f"{roi_name}_nuclei")

    # Add the ROIs as labels into Napari
    viewer.add_labels(user_roi, name=f"{roi_name}_ROI", opacity=0.4)

    # Simulate a cytoplasm by growing the nuclei_labels
    if location == "cytoplasm":
        
        nuclei_labels = cle.dilate_labels(nuclei_labels, radius=cytoplasm_dilation_radius)
        nuclei_labels = cle.pull(nuclei_labels)

    # Select marker positive nuclei
    nuclei_and_marker, eroded_nuclei_and_marker, marker_mip, processed_region_labels = segment_marker_positive_nuclei (nuclei_labels, marker_img, marker_channel_threshold, erosion_factor)
    viewer.add_image(nuclei_and_marker, name=f"{roi_name}_marker_nuclei_coloc")
    viewer.add_image(eroded_nuclei_and_marker, name=f"{roi_name}_marker_nuclei_eroded")
    viewer.add_labels(processed_region_labels, name=f"{roi_name}_marker+_nuclei")

    # Extract your information of interest
    total_nuclei = len(np.unique(nuclei_labels)) - 1
    marker_pos_nuclei = len(np.unique(processed_region_labels)) - 1

    # Calculate "%_marker+_cells" and avoid division by zero errors
    try:
        perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
    except ZeroDivisionError:
        perc_marker_pos_cells = 0

    # Create a dictionary containing all extracted info per masked image
    stats_dict = {
                "filename": filename,
                "ROI": roi_name,
                "total_nuclei": total_nuclei,
                "marker+_nuclei": marker_pos_nuclei,
                "%_marker+_cells": perc_marker_pos_cells,
                "nuclei_ch": nuclei_channel,
                "marker_ch": marker_channel,
                "marker_int_threshold": marker_channel_threshold,
                "marker_location":location,
                "cytoplasm_dilation":cytoplasm_dilation_radius,
                "erosion_factor": erosion_factor,
                "cellpose_nuclei_diameter": cellpose_nuclei_diameter,
                "gaussian_sigma": gaussian_sigma,
                "slicing_factor": slicing_factor
                }

    # Append the current data point to the stats_list
    stats.append(stats_dict)    

100%|██████████| 3/3 [00:54<00:00, 18.32s/it]


<h3>Data saving</h3>


In [16]:
# Define output folder for results
results_folder = "./results/"

# Create the necessary folder structure if it does not exist
try:
    os.mkdir(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# Transform into a dataframe to store it as .csv later
df = pd.DataFrame(stats)

# Define the .csv path
csv_path = "./results/marker_+_label_coloc_2D.csv"

# Append to the .csv with new data points each round
df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df

Output folder created: ./results/


,filename,ROI,total_nuclei,marker+_nuclei,%_marker+_cells,nuclei_ch,marker_ch,marker_int_threshold,marker_location,cytoplasm_dilation,erosion_factor,cellpose_nuclei_diameter,gaussian_sigma,slicing_factor
0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,ALL,5054,2782,55.045509,3,1,40,nucleus,2,4,NaN,0,NaN
1,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,1223,1103,90.188062,3,1,40,nucleus,2,4,NaN,0,NaN
2,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,1912,1473,77.039749,3,1,40,nucleus,2,4,NaN,0,NaN
